# *_FRIENDSTER - Social Network_*

> Friendster est un reseau de type petit monde qui mélange cercle d'ami et contact reseau.
> La densité de connextion est très élévée il faut calculer la betweeness centralite pour retirer l'arete centrale

_Alternative_

> Si ordre de complexité trop eleve pour ma machine alors approximation à faire :
> on considere le degres d'un noeud elevé comme etant un des sommets de l'arete centrale. 
> Je cherche parmis les noeuds de plus fort degres l'arete si elle n'existe pas alors on cherche dans le lot ayant un peu moins de degres et ainsi de suite jusqu'au plus petit degrès.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Row
import re

In [82]:
sc.getConf().getAll()

[('spark.dynamicAllocation.minExecutors', '1'), ('spark.shuffle.service.enabled', 'true'), ('spark.jars', 'file:/usr/local/share/jupyter/kernels/apache_toree_pyspark/lib/toree-assembly-0.2.0.dev1-incubating-SNAPSHOT.jar'), ('spark.app.id', 'local-1531471674611'), ('spark.executor.id', 'driver'), ('spark.repl.class.uri', 'spark://192.168.254.21:44187/classes'), ('spark.dynamicAllocation.maxExecutors', '5'), ('spark.repl.class.outputDir', '/tmp/spark-fc263e40-7870-4d15-837a-4f8953a0c0c6/repl-79443874-7220-4ce4-a4e7-c8a62cc61092'), ('spark.driver.port', '44187'), ('spark.app.name', 'Friendster-mapreduce-by-key'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.deployMode', 'client'), ('spark.dynamicAllocation.enabled', 'true'), ('spark.driver.host', '192.168.254.21')]

In [83]:
conf = SparkConf().setAll([('spark.app.name', 'Friendster-mapreduce-by-key'),("spark.dynamicAllocation.enabled", "true"),("spark.dynamicAllocation.minExecutors", 1), ("spark.dynamicAllocation.maxExecutors", 5), ("spark.shuffle.service.enabled", "true") ])
sc.stop()
sc = SparkContext(conf=conf)

In [5]:
sc.getConf().getAll()

[('spark.dynamicAllocation.minExecutors', '1'), ('spark.shuffle.service.enabled', 'true'), ('spark.jars', 'file:/usr/local/share/jupyter/kernels/apache_toree_pyspark/lib/toree-assembly-0.2.0.dev1-incubating-SNAPSHOT.jar'), ('spark.app.id', 'local-1531471674611'), ('spark.executor.id', 'driver'), ('spark.repl.class.uri', 'spark://192.168.254.21:44187/classes'), ('spark.dynamicAllocation.maxExecutors', '5'), ('spark.repl.class.outputDir', '/tmp/spark-fc263e40-7870-4d15-837a-4f8953a0c0c6/repl-79443874-7220-4ce4-a4e7-c8a62cc61092'), ('spark.driver.port', '44187'), ('spark.app.name', 'Friendster-mapreduce-by-key'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.deployMode', 'client'), ('spark.dynamicAllocation.enabled', 'true'), ('spark.driver.host', '192.168.254.21')]

In [6]:
sc

<SparkContext master=local[*] appName=Friendster-mapreduce-by-key>

In [7]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

# Création du graphe 

> **Hypothèse : ** Les liens entre les utilisateurs sont des liens 
    orientés entre utilisateur influant et le suiveur
    

In [30]:
import org.apache.spark.graphx.GraphLoader
//#créer un fichier avec les noeuds et leur attribut (quel format ?)
//#Chargement des utilisateurs (nodes) et transformer les lignes d'utilisateur avec id and nombre de suiveur
val users = (sc.textFile("/home/pb19121/datagraphx/friendsterusers.txt")
  .map(line => line.split(",")).map( parts => (parts.head.toLong, parts.tail) ))

// Parse the edge data which is already in userId -> userId format
val followerGraph = GraphLoader.edgeListFile(sc, "/home/pb19121/datagraphx/friendsterfollowers.txt")

//# Attach the user attributes
val graph = followerGraph.outerJoinVertices(users) {
  case (uid, deg, Some(attrList)) => attrList
 // # Some users may not have attributes so we set them as empty
  case (uid, deg, None) => Array.empty[String]
}

In [30]:
//Ajouter les attributs à partir des calculs de degres, indegree et outdegree dans le fichiers des users
import org.apache.spark.graphx.GraphLoader

val users = (sc.textFile("/home/pb19121/datagraphx/friendsterusers.txt")
  .map(line => line.split(",")).map( parts => (parts.head.toLong, parts.tail) ))

// Parse the edge data which is already in userId -> userId format
val edgesGraph = GraphLoader.edgeListFile(sc, "/home/pb19121/datagraphx/friendsteredges.txt")

//# Attach the user attributes
val graph = edgesGraph.outerJoinVertices(users) {
  case (uid, deg, Some(attrList)) => attrList
 // # Some users may not have attributes so we set them as empty
  case (uid, deg, None) => Array.empty[String]
}

In [105]:
graph

org.apache.spark.graphx.impl.GraphImpl@4eea6d61

In [ ]:
// Restrict the graph to users with usernames and names
val subgraph = graph.subgraph(vpred = (vid, attr) => attr.size == 2)

// Compute the PageRank
val pagerankGraph = subgraph.pageRank(0.001)

// Get the attributes of the top pagerank users
val userInfoWithPageRank = subgraph.outerJoinVertices(pagerankGraph.vertices) {
  case (uid, attrList, Some(pr)) => (pr, attrList.toList)
  case (uid, attrList, None) => (0.0, attrList.toList)
}

In [33]:
println(userInfoWithPageRank.vertices.top(10)(Ordering.by(_._2._1)).mkString("\n"))

(1,(8019.989012755636,List(N1, n1)))
(2,(3473.697419631351,List(N2, n2)))
(82,(1691.8315267036721,List(N82, n82)))
(11,(1038.692904959271,List(N11, n11)))
(3,(1033.6143079036847,List(N3, n3)))
(13,(920.921867922792,List(N13, n13)))
(4,(670.6766422815854,List(N4, n4)))
(54,(648.4665619563988,List(N54, n54)))
(5,(546.9728220426837,List(N5, n5)))
(20,(540.7584562672422,List(N20, n20)))


In [52]:
graph.numEdges

981920

# Distribution de degrés

In [56]:
// Define a reduce operation to compute the highest degree vertex
def max(a: (VertexId, Int), b: (VertexId, Int)): (VertexId, Int) = {
  if (a._2 > b._2) a else b
}
// Compute the max degrees
val maxInDegree: (VertexId, Int)  = graph.inDegrees.reduce(max)
val maxOutDegree: (VertexId, Int) = graph.outDegrees.reduce(max)
val maxDegrees: (VertexId, Int)   = graph.degrees.reduce(max)

In [71]:
print (" max indegree : ")
maxInDegree

 max indegree : 

(3687,620)

In [72]:
print (" max outdegree : ")
maxOutDegree

 max outdegree : 

(111058,165)

In [73]:
print (" max degree : ")
maxDegrees

 max degree : 

(3687,621)

In [76]:
val degrees: VertexRDD[Int] = graph.degrees

In [113]:
degrees.take(5).foreach(println)

(108150,3)
(129434,5)
(91902,16)
(68522,14)
(23776,4)


In [75]:
val indegrees: VertexRDD[Int] = graph.inDegrees

In [74]:
val outdegrees: VertexRDD[Int] = graph.outDegrees

In [54]:
degrees.map(_._1).stats()

(count: 100199, mean: 69079.757972, stdev: 41109.577764, max: 151837.000000, min: 1.000000)

In [77]:
indegrees.map(_._1).stats()

(count: 79262, mean: 63332.714163, stdev: 39503.759873, max: 151820.000000, min: 1.000000)

In [78]:
outdegrees.map(_._1).stats()

(count: 100198, mean: 69080.447394, stdev: 41109.203660, max: 151837.000000, min: 2.000000)

In [55]:
degrees.map(_._2).stats()

(count: 100199, mean: 19.599397, stdev: 24.609730, max: 621.000000, min: 1.000000)

In [79]:
indegrees.map(_._2).stats()

(count: 79262, mean: 12.388282, stdev: 20.920199, max: 620.000000, min: 1.000000)

In [80]:
outdegrees.map(_._2).stats()

(count: 100198, mean: 9.799796, stdev: 11.073388, max: 165.000000, min: 1.000000)

# La “longueur moyenne des (plus courts) chemins” : 

# Coeficient de clustering : 

In [39]:
val triplets = graph.triplets

In [40]:
triplets.count()

981920

In [99]:
triplets.take(5).foreach(println)

((2,[Ljava.lang.String;@14465c0b),(1,[Ljava.lang.String;@4d2963b0),1)
((3,[Ljava.lang.String;@214b735f),(1,[Ljava.lang.String;@4d2963b0),1)
((4,[Ljava.lang.String;@3ca7f4b8),(1,[Ljava.lang.String;@4d2963b0),1)
((4,[Ljava.lang.String;@3ca7f4b8),(2,[Ljava.lang.String;@14465c0b),1)
((5,[Ljava.lang.String;@1aa74123),(1,[Ljava.lang.String;@4d2963b0),1)


In [90]:
val triCounts = graph.triangleCount()

In [91]:
triCounts.vertices.map(x => x._2).stats()

(count: 100199, mean: 63.479735, stdev: 140.725195, max: 4142.000000, min: 0.000000)

In [92]:
val maxTrisGraph = triCounts.degrees.mapValues(d => d * (d - 1) / 2.0)

In [93]:
maxTrisGraph

VertexRDDImpl[1895] at RDD at VertexRDD.scala:57

In [94]:
val clusterCoefGraph = triCounts.vertices.innerJoin(maxTrisGraph) {
  (vertexId, triCount, maxTris) => {
    if (maxTris == 0) 0
    else triCount / maxTris
  }
}

In [96]:
clusterCoefGraph.map(_._2).sum() / triCounts.vertices.count()

0.2672521165006904

> **Notes : **
- on obtient 26% caractéristique des réseaux de petits mondes
- à comparer avec un graphique similaire (club de karate de zachary ou aléatoire)


In [106]:
val connectedComponentGraph: Graph[VertexId, Int] = graph.connectedComponents()

In [107]:
def sortedConnectedComponents(connectedComponents: Graph[VertexId, _]): Seq[(VertexId, Long)] = {
  val componentCounts = connectedComponents.vertices.map(_._2).countByValue
  componentCounts.toSeq.sortBy(_._2).reverse
}

In [108]:
val componentCounts = sortedConnectedComponents( connectedComponentGraph)

In [109]:
componentCounts.size

1

In [110]:
componentCounts.take(10).foreach(println)

(1,100199)
